# Lab 4 Folium

## Question 1

Write a script that calculates the distance between the UWT and UPS points used above in **any** linear unit.
#### Bonus: Calculate the distance in feet

In [83]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium

data = [['UWT', 47.246613, -122.438813], ['UPS',47.263660,-122.482941]]

panda = pandas.DataFrame(data, columns=['name','lat','long'])
print(panda)

#Here I use the zip function to basically... 'zip' two list items into one.
#Find out more here: https://docs.python.org/2/library/functions.html#zip
coords = zip(panda['long'], panda['lat'])

#I am going to collapse a for loop into a single line here, watch how

geometry = [shapely.geometry.Point(c) for c in coords]
#Shapely is a library with the point data type GeoPandas uses for coordinates

#Now, I'm going to make that geometry into a GeoSeries
geolist = geopandas.GeoSeries(geometry)

#Ok, let's put it all together.
geoPoints = geopandas.GeoDataFrame(
        panda,
        geometry=geolist)

#And now, TO A WEB MAP
geoPoints.crs=fiona.crs.from_epsg(4236) #What am I doing here? Why? UTM Projection

#change projection to get accurate distance measurement
geoPoints.to_crs(epsg=3310,inplace=True)

map = folium.Map(
                location=[47.2529, -122.4443],
                zoom_start=11,
                tiles='Stamen Toner')

twopoints = folium.features.GeoJson(geoPoints.to_json()).add_to(map)

#distance measure in both meters (default) and feet
distance = geoPoints.distance(geoPoints.shift())
d_ft = distance / 0.3048
print(distance, 'meters')
print(d_ft, 'feet')

  name        lat        long
0  UWT  47.246613 -122.438813
1  UPS  47.263660 -122.482941
0            NaN
1    3872.180527
dtype: float64 meters
0             NaN
1    12704.004355
dtype: float64 feet


## Question 2

Take the natural earth cities layer that we worked with earlier and convert it into a folium map.

In [84]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium

world = geopandas.read_file(
    geopandas.datasets.get_path('naturalearth_lowres')
)
world['gdp_per_cap']= world.gdp_md_est / world.pop_est *1000000

map = folium.Map(
                location=[14.5994, -28.6731],
                zoom_start=2,
                tiles='cartodbpositron')

world = folium.features.GeoJson(world.to_json()).add_to(map)

map

### Question 3: (Not) going home?

I want you to produce a 'random walk.' It's not quite random due to the constraints I'm going to put on it *and* due to the pseudorandom nature of generating the numbers, but it's going to force you to think through the basic ideas of this algorithm.

You are going to write a 'simulation' of a random walk wherein our walker takes up to 10,000 steps to try to find their way home. You'll then calculate how far they are from where they started and how far they are from their destination. 

Specifically, your script will:

1. Begin at (-122, 47)
2. Have a home at (-71, 42)
3. Your walker moves random following these rules: Each step is one unit wrong and takes place in one of four directions. In other words, with each step your walker will increase or decrease x or y by 1.
4. After 10,000 steps or arriving at (-71, 42) the walk ends immediately. 
5. When the walk ends, you will print out: total steps taken, end location, distance from start location, distance from home location.

**hint**: You can approach this by creating a class or two (with methods) or by writing functions and using a loop. The former is more expandable (and might be useful later if we return to this method), the latter is likely more straightforward. (Really, you should probably be using a generator, but that stuff gets weird fast in my opinion).

In [15]:
import numpy 
import pylab 
import random 
  
'''
#MADE A SWEET PLOT MAP!!!

#number of steps 
n = 10000
  
#creating x and y coordinate 
#of size equals to the number of size and filled up with 0's 
x = numpy.zeros(n) 
y = numpy.zeros(n) 
  
#random variables 
for i in range(1, n): 
    val = random.randint(1, 4) 
    if val == 1: 
        x[i] = x[i - 1] + 1
        y[i] = y[i - 1] 
    elif val == 2: 
        x[i] = x[i - 1] - 1
        y[i] = y[i - 1] 
    elif val == 3: 
        x[i] = x[i - 1] 
        y[i] = y[i - 1] + 1
    else: 
        x[i] = x[i - 1] 
        y[i] = y[i - 1] - 1
      
#plotting stuff
pylab.title("Random Walk ($n = " + str(n) + "$ steps)") 
pylab.plot(x, y) 
pylab.savefig("rand_walk"+str(n)+".png",bbox_inches="tight",dpi=1000) 
pylab.show()'''

'''
# Closer
def walker (n):
   x, y = -122, 47
   for i in range (n):
       (wx, wy) = random.choice ([(0, 1), (0, -1), (1, 0), (-1, 0)])
       x += wx
       y += wy
   return (x, y)

number_of_steps = 10000

for walk_stop in range (-71, 42):
   for i in range (number_of_steps):
       (x, y) = walker (walk_stop)
       distance = abs (x) + abs (y)
# stackoverflow, found video'''

# From video 'A Random Walk & Monte Carlo Simulation' - YouTube
def random_walk(n):
    '''Return coordiantes after 'n' block random walk'''
    x, y = -122, 42
    # y = 0
    for i in range(n):
        (dx, dy) = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
        x = x+dx
        y = y+dy
    return(x, y)
    
for i in range(1):
    walk = random_walk(10000)
    distance = (abs(walk[0] +71)) + (abs(walk[1] - 42))
    print('End coordinates:', walk)
    print('Distance from origin:', abs(walk[0]) + abs(walk[1]))
    # fellow MSGT student, they know who they are
    print ('Distance from home:', distance)

End coordinates: (-152, -30)
Distance from origin: 182
Distance from home: 153


## Bonus Question 1: Well, where did we go?

Create an interactive web map (so the output is  leaflet map through folium) to put your random walker onto a world map.

In [19]:
import shapely, shapely.geometry, fiona, fiona.crs, pandas, folium, numpy, pylab, random 

def random_walk(n):
    # Return coordiantes after 'n' block random walk
    x, y = -122, 42
    # y = 0
    for i in range(n):
        (dx, dy) = random.choice([(0, 1), (0, -1), (1, 0), (-1, 0)])
        x = x+dx
        y = y+dy
    return(x, y)
    
for i in range(1):
    walk = random_walk(10000)
    distance = (abs(walk[0] +71)) + (abs(walk[1] - 42))
    #print(walk, 'Distance from origin:', abs(walk[0]) + abs(walk[1]))
    # fellow MSGT student, they know who they are
    #print ('Distance from home:', distance)

path = walk

m = folium.Map(
    location=[39, -98],
    zoom_start=2,
    tiles='Mapbox Control Room'
)

folium.Marker([47, -121], icon=folium.Icon(icon='road', color='green'), popup='<i>Start</i>').add_to(m)
folium.Marker([42, -71], icon=folium.Icon(icon='home', color='green'), popup='<b>Home</b>').add_to(m)
#folium.Marker([wakk], icon=folium.Icon(icon='user', color='green'), popup='Finish').add_to(m)
#NO! I feel like it is close
#folium.features.GeoJson(data=path).add_to(m)

m